In [0]:
import torch
from torchtext import data

SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize='spacy')
LABEL = data.LabelField(dtype=torch.float)

In [3]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

aclImdb_v1.tar.gz:   0%|          | 147k/84.1M [00:00<01:04, 1.30MB/s]

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:01<00:00, 53.8MB/s]


In [4]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 25000
Number of testing examples: 25000


In [5]:

print(vars(train_data.examples[0]))

{'text': ['One', 'of', 'the', 'most', 'timely', 'and', 'engrossing', 'documentaries', ',', 'you', "'ll", 'ever', 'watch', '.', 'While', 'the', 'story', 'takes', 'place', 'in', 'the', 'Venezuelan', 'capital', 'of', 'Caracas', ',', 'it', 'provides', 'an', 'intimate', 'look', 'into', 'political', 'dynamics', ',', 'that', 'prevail', 'throughout', 'the', 'western', 'Hemisphere', '.', 'While', 'essentially', 'another', 'chapter', 'in', 'the', 'story', 'of', 'the', '"', 'U.S.', 'backed', ',', 'Latin', 'American', 'coup', '"', ',', 'this', 'film', 'chronicles', 'in', 'real', '-', 'time', ',', 'what', 'can', 'happen', 'when', 'the', 'poorest', 'people', ',', 'are', 'armed', 'with', 'unity', ',', 'political', 'savvy', ',', 'and', 'courage', '!', '<', 'br', '/><br', '/>The', 'political', 'insights', 'offered', 'by', 'this', 'film', 'are', 'invaluable', '.', 'One', 'gets', 'clear', 'examples', 'of', 'the', 'private', 'media', ',', 'as', 'a', 'formidable', 'force', 'for', 'mass', 'deception', 'and'

In [0]:
import random

train_data, valid_data = train_data.split(random_state=random.seed(SEED))

In [7]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [0]:
TEXT.build_vocab(train_data, max_size=25000)
LABEL.build_vocab(train_data)

In [9]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [10]:

print(TEXT.vocab.freqs.most_common(20))

[('the', 201942), (',', 191425), ('.', 165181), ('and', 108820), ('a', 108665), ('of', 100148), ('to', 93321), ('is', 75836), ('in', 61069), ('I', 54380), ('it', 53558), ('that', 48938), ('"', 43613), ("'s", 42927), ('this', 42329), ('-', 36420), ('/><br', 35416), ('was', 34933), ('as', 30098), ('with', 29758)]


In [11]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'and', 'a', 'of', 'to', 'is']


In [12]:
print(LABEL.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fd293c07048>, {'neg': 0, 'pos': 1})


In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size=BATCH_SIZE,
    device=device)

In [0]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):

        #x = [sent len, batch size]
        
        embedded = self.embedding(x)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [0]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1

model = RNN(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM)

In [0]:
import torch.optim as optim

optimizer = optim.SGD(model.parameters(), lr=1e-3)

In [0]:

criterion = nn.BCEWithLogitsLoss()

In [0]:
model = model.to(device)
criterion = criterion.to(device)

In [0]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum()/len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
                
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [24]:
N_EPOCHS = 5

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

| Epoch: 01 | Train Loss: 0.694 | Train Acc: 50.27% | Val. Loss: 0.699 | Val. Acc: 49.39% |
| Epoch: 02 | Train Loss: 0.693 | Train Acc: 49.73% | Val. Loss: 0.699 | Val. Acc: 49.11% |
| Epoch: 03 | Train Loss: 0.693 | Train Acc: 49.69% | Val. Loss: 0.699 | Val. Acc: 50.42% |
| Epoch: 04 | Train Loss: 0.693 | Train Acc: 50.02% | Val. Loss: 0.698 | Val. Acc: 48.60% |
| Epoch: 05 | Train Loss: 0.693 | Train Acc: 50.07% | Val. Loss: 0.699 | Val. Acc: 50.43% |


In [25]:
test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}% |')

| Test Loss: 0.711 | Test Acc: 46.76% |
